<h1>Mental Health Model Trainer Notebook</h1>
<h3>Training the model based on data scraped from the US Census Bureau</h3>
<h2><b>IMPORTANT:</b><br><span style="color:red;">This product uses the Census Bureau Data API but is not endorsed or certified by the Census Bureau.</span></h2>

<ol>
    <li>Split the data into training, test, and validation sets with random seed <b>123</b></li>
    <ul>
        <li>4 fold cross validation - 80% of the data with training = 60%, validation = 20%</li>
        <li>Test = 20%</li>
    </ul>
    <li>Train the model based on broadband internet only, normalize y value, minimize MSE</li>
    <li>Use LASSO regularization to eliminate unecessary features from the theta vector</li>
    <li>Retrain the model with demographic features added in</li>
    <li>Retrain the model with economic features added in</li>
    <li>Retrain the model with social features added in</li>
</ol>

In [1]:
#Imports
#Data manipulation/training
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

#Visualization
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt